<a href="https://colab.research.google.com/github/ramprakashyadav/OCR/blob/main/invoice_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [13]:
!pip install tensorflow==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Thu Jul 14 15:34:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%cd /content/gdrive/My Drive/programs/Datasets/

/content/gdrive/My Drive/programs/Datasets


In [5]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [7]:
!pip install keras==2.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 10.8 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0+zzzcolab20220506150900 requires keras<2.8,>=2.7.0rc0, but you have keras 2.6.0 which is incompatible.
tensorflow 2.7.0+zzzcolab20220506150900 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 2.9.0 which is incompatible.
tensorflow-text 2.9.0 requires tensorflow<2.10,>=2.9.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.7.0+zzzcolab20220506150900 which is incompatible.
tensorflow-gpu 2.9.1 requires keras<2.10.0,>=2.9.0rc0, but y

In [8]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [9]:
IMAGE_SIZE = [224, 224]

train_path = '/content/gdrive/My Drive/programs/Datasets/train'
valid_path = '/content/gdrive/My Drive/programs/Datasets/test'

In [10]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
for layer in inception.layers:
    layer.trainable = False

In [12]:
 # useful for getting number of output classes
folders = glob('/content/gdrive/My Drive/programs/Datasets/train/*')

In [13]:
folders

['/content/gdrive/My Drive/programs/Datasets/train/vero moda',
 '/content/gdrive/My Drive/programs/Datasets/train/tueren',
 '/content/gdrive/My Drive/programs/Datasets/train/only retail',
 '/content/gdrive/My Drive/programs/Datasets/train/kobelco',
 '/content/gdrive/My Drive/programs/Datasets/train/supermilk',
 '/content/gdrive/My Drive/programs/Datasets/train/olom',
 '/content/gdrive/My Drive/programs/Datasets/train/redbull invoice',
 '/content/gdrive/My Drive/programs/Datasets/train/redbull challan',
 '/content/gdrive/My Drive/programs/Datasets/train/soprema',
 '/content/gdrive/My Drive/programs/Datasets/train/overseas',
 '/content/gdrive/My Drive/programs/Datasets/train/kelly',
 '/content/gdrive/My Drive/programs/Datasets/train/global',
 '/content/gdrive/My Drive/programs/Datasets/train/kaya',
 '/content/gdrive/My Drive/programs/Datasets/train/fujitsu challan',
 '/content/gdrive/My Drive/programs/Datasets/train/fujitsu invoice',
 '/content/gdrive/My Drive/programs/Datasets/train/hoc

In [14]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [15]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [16]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/gdrive/My Drive/programs/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 593 images belonging to 20 classes.


In [20]:
test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/programs/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 306 images belonging to 20 classes.


In [22]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# r = model.fit_generator(training_set, #training data to fit
#                         steps_per_epoch=8000, # Data in training set
#                         epochs=10, # No of epochs to run
#                         validation_data=test_set, # Test or validation set
#                         validation_steps=2000 # no of data point for validation
#                         )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
19/19 [==============================] - 42s 2s/step - loss: 0.4582 - accuracy: 0.9477 - val_loss: 0.2783 - val_accuracy: 0.9706
Epoch 2/50
19/19 [==============================] - 37s 2s/step - loss: 0.0799 - accuracy: 0.9815 - val_loss: 0.8587 - val_accuracy: 0.9020
Epoch 3/50
19/19 [==============================] - 37s 2s/step - loss: 0.1305 - accuracy: 0.9764 - val_loss: 0.2743 - val_accuracy: 0.9641
Epoch 4/50
19/19 [==============================] - 37s 2s/step - loss: 0.1343 - accuracy: 0.9916 - val_loss: 0.3107 - val_accuracy: 0.9379
Epoch 5/50
19/19 [==============================] - 38s 2s/step - loss: 0.1432 - accuracy: 0.9815 - val_loss: 0.5654 - val_accuracy: 0.9314
Epoch 6/50
19/19 [==============================] - 37s 2s/step - loss: 0.0577 - accuracy: 0.9865 - val_loss: 0.5872 - val_accuracy: 0.9608
Epoch 7/50
19/19 [==============================] - 37s 2s/step - loss: 0.1348 - accuracy: 0.9798 - val_loss: 0.9139 - val_accuracy: 0.9281
Epoch 8/50
19/19 [==

In [62]:
!pip install tensorflow==2.7.0
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 665.5 MB 24 kB/s 
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
    Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
      Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.


In [23]:
from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')